<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1be953c3287a9da09255edde68fb742758c1d1caed26a84908e0eeae5803558f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 13:02:16
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -31.40 K (-0.22%)
Current PnL: -22.43 L (-14.7%)
CY Booked + Current PnL: -8.54 L (-5.59%)
-------------------
Total profit:  1.67 L
Total loss:  -24.10 L
-------------------
Total Booked + Current PnL: 18.44 L (14.68%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.69%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.84 L (61.27%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,0.08,-14.04,16.45,0.09,14250.0,-14153.0,86624.0,101.07,50.0,M-SC,3.55,253.0,-0.99,0.60,13.80,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-3.60,-10.53,11.79,0.02,18414.0,-18375.0,156184.0,138.28,62.0,M-SC,9.05,234.0,-1.00,1.09,35.85,OX40N,NTT,PAINTS
83,VOLTAS,1530.0,-0.50,7.97,10.86,19.69,22482.0,15273.0,207015.0,-2.60,51.0,X-MC,3.35,78.0,0.68,1.44,15.30,XY25,NTT,AC
43,ITC,452.0,0.17,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,44.0,X-LC,1.02,5.0,-0.13,1.37,3.94,X40,NTT,FMCG
52,MEDANTA,1486.0,-0.56,4.89,18.29,24.08,24354.0,6207.0,133157.0,-3.56,49.0,X-SC,5.54,89.0,0.25,0.93,25.46,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,3.94,3.51,27.93,32.42,73400.0,8904.0,262800.0,-14.94,51.0,X-SC,7.54,85.0,0.12,1.83,19.14,X40,ATH,FMCG
67,SIS,528.00,2.62,-20.85,53.20,21.26,46755.0,-23146.0,87886.0,2077.72,61.0,H-SC,3.78,166.0,-0.50,0.61,18.74,OX40N,NTT,MISC
58,RAJOOENG,143.10,2.59,-25.48,87.35,39.61,66718.0,-26120.0,76380.0,-49.69,38.0,H-SC,20.51,135.0,-0.39,0.53,7.65,AR,ATH,MISC
59,RECLTD,446.00,2.05,-1.54,22.63,20.74,45268.0,-3135.0,200035.0,28.81,50.0,M-MC,5.30,189.0,-0.07,1.40,5.21,XY25,NTT,FINANCE
21,DEN,75.00,1.84,-39.29,134.01,42.07,68720.0,-33184.0,51280.0,81.35,47.0,M-SC,8.15,235.0,-0.48,0.36,8.90,AR,NTT,ENTERTAINMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-10.65,-14.01,111.56,81.92,110128.0,-16082.0,98716.0,-49.60,23.0,M-SC,3.34,236.0,-0.15,0.69,16.86,XR,NTT,DURABLES
39,INDIGOPNTS,1408.0,-3.60,-10.53,11.79,0.02,18414.0,-18375.0,156184.0,138.28,62.0,M-SC,9.05,234.0,-1.00,1.09,35.85,OX40N,NTT,PAINTS
28,GREENPANEL,537.0,-2.23,-32.70,112.67,43.14,138832.0,-59858.0,123220.0,175.84,32.0,M-SC,5.86,240.0,-0.43,0.86,15.91,XY24,NTT,MISC
70,SURYODAY,216.0,-1.88,-19.80,53.41,23.03,76705.0,-35455.0,143616.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.50,XR,NTT,BANKS
44,JCHAC,2282.0,-1.69,-26.08,35.27,-0.01,33320.0,-33333.0,94472.0,19226.09,22.0,M-SC,1.52,233.0,-1.00,0.66,6.98,OX40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.13,0.67,105.35,106.72,130800.0,822.0,124158.0,-50.25,40.0,H-SC,3.09,139.0,0.01,0.87,24.2,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.40,-4.58,117.90,107.91,166093.0,-6768.0,140876.0,-22.17,37.0,M-SC,10.57,216.0,-0.04,0.98,0.74,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-3.60,-10.53,11.79,0.02,18414.0,-18375.0,156184.0,138.28,62.0,M-SC,9.05,234.0,-1.00,1.09,35.85,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.08,-14.04,16.45,0.09,14250.0,-14153.0,86624.0,101.07,50.0,M-SC,3.55,253.0,-0.99,0.60,13.80,OX40N,NTT,DURABLES
67,SIS,528.00,2.62,-20.85,53.20,21.26,46755.0,-23146.0,87886.0,2077.72,61.0,H-SC,3.78,166.0,-0.50,0.61,18.74,OX40N,NTT,MISC
47,KANSAINER,340.00,0.76,-21.30,44.19,13.47,93780.0,-57447.0,212220.0,-67.73,37.0,H-SC,1.48,158.0,-0.61,1.48,7.91,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.66,1.94,68.99,72.27,119170.0,3287.0,172735.0,-12.43,57.0,M-LC,3.89,99.0,0.03,1.20,8.23,XR,NTT,IT
38,INDIAMART,4810.62,0.13,0.67,105.35,106.72,130800.0,822.0,124158.0,-50.25,40.0,H-SC,3.09,139.0,0.01,0.87,24.20,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.66,1.94,68.99,72.27,119170.0,3287.0,172735.0,-12.43,57.0,M-LC,3.89,99.0,0.03,1.20,8.23,XR,NTT,IT
38,INDIAMART,4810.62,0.13,0.67,105.35,106.72,130800.0,822.0,124158.0,-50.25,40.0,H-SC,3.09,139.0,0.01,0.87,24.20,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.47,-1.29,37.40,35.63,77372.0,-2696.0,206878.0,-15.41,41.0,H-MC,3.97,119.0,-0.03,1.44,15.17,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.40,-4.58,117.90,107.91,166093.0,-6768.0,140876.0,-22.17,37.0,M-SC,10.57,216.0,-0.04,0.98,0.74,OX40N,ATH,CABLES
37,IEX,219.00,-0.55,-4.32,55.34,48.63,105871.0,-8644.0,191310.0,-36.24,52.0,H-SC,15.62,136.0,-0.08,1.33,6.73,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.34,-35.26,108.83,35.20,90663.0,-45363.0,83307.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.34,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.07,-14.93,67.13,42.18,157853.0,-41265.0,235145.0,-23.08,29.0,X-LC,1.47,3.0,-0.26,1.64,1.86,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.46,-8.70,26.20,15.23,47926.0,-17422.0,182925.0,-21.89,31.0,X-MC,6.91,66.0,-0.36,1.28,20.75,X40N,NTT,REALTY
55,PGHH,17907.65,0.07,-4.50,40.07,33.76,76844.0,-9045.0,191775.0,-32.64,33.0,X-MC,3.72,57.0,-0.12,1.34,0.46,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.65,-13.56,71.74,48.45,124336.0,-27191.0,173315.0,-19.87,35.0,X-MC,7.30,64.0,-0.22,1.21,0.69,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.17,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,44.0,X-LC,1.02,5.0,-0.13,1.37,3.94,X40,NTT,FMCG
76,TMPV,600.0,-0.07,-14.93,67.13,42.18,157853.0,-41265.0,235145.0,-23.08,29.0,X-LC,1.47,3.0,-0.26,1.64,1.86,XY24,NTT,AUTO
3,ACC,3906.0,-0.62,-21.29,108.55,64.15,203303.0,-50661.0,187290.0,-54.13,54.0,X-SC,1.48,82.0,-0.25,1.31,5.06,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.51,3.63,26.38,30.97,48316.0,6416.0,183154.0,-20.80,58.0,X-MC,1.83,75.0,0.13,1.28,16.65,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-0.81,-11.55,41.90,25.51,68967.0,-21495.0,164600.0,0.93,63.0,H-LC,1.88,49.0,-0.31,1.15,21.30,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.34,-35.26,108.83,35.20,90663.0,-45363.0,83307.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.34,X40,NTT,FOOTWEAR
55,PGHH,17907.65,0.07,-4.50,40.07,33.76,76844.0,-9045.0,191775.0,-32.64,33.0,X-MC,3.72,57.0,-0.12,1.34,0.46,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.65,-13.56,71.74,48.45,124336.0,-27191.0,173315.0,-19.87,35.0,X-MC,7.30,64.0,-0.22,1.21,0.69,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.21,-4.92,32.59,26.07,50736.0,-8060.0,155680.0,-29.94,39.0,X-MC,9.37,55.0,-0.16,1.09,1.00,X40,ATH,APPARELS
19,COLPAL,3726.84,-0.91,-17.79,72.12,41.51,156154.0,-46845.0,216520.0,-2.12,42.0,X-MC,7.18,61.0,-0.30,1.51,1.61,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.41,-27.90,98.13,42.84,45990.0,-18140.0,46866.0,-52.62,41.0,X-SC,37.57,83.0,-0.39,0.33,1.98,XY24,NTT,MISC
60,RELAXO,1176.00,-0.58,-46.90,186.83,52.31,144009.0,-68080.0,77080.0,-43.47,38.0,X-SC,4.44,91.0,-0.47,0.54,3.18,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.34,-35.26,108.83,35.20,90663.0,-45363.0,83307.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.34,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.59,-5.35,28.46,21.58,37750.0,-7497.0,132642.0,-52.25,37.0,X-MC,6.56,56.0,-0.20,0.93,17.43,X40N,ATH,IT
52,MEDANTA,1486.00,-0.56,4.89,18.29,24.08,24354.0,6207.0,133157.0,-3.56,49.0,X-SC,5.54,89.0,0.25,0.93,25.46,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-1.03,-14.95,40.25,19.28,118431.0,-51717.0,294239.0,-25.89,57.0,X-LC,7.24,1.0,-0.44,2.05,8.78,X40,ATH,IT
41,INFY,2275.00,0.01,6.06,46.02,54.87,152719.0,18966.0,331854.0,-17.44,61.0,X-LC,2.63,2.0,0.12,2.31,14.85,X40,BTT,IT
76,TMPV,600.00,-0.07,-14.93,67.13,42.18,157853.0,-41265.0,235145.0,-23.08,29.0,X-LC,1.47,3.0,-0.26,1.64,1.86,XY24,NTT,AUTO
82,VBL,671.64,0.11,-5.87,44.13,35.67,131202.0,-18534.0,297308.0,-17.02,54.0,X-LC,5.77,4.0,-0.14,2.07,7.17,X40N,ATH,FMCG
43,ITC,452.00,0.17,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,44.0,X-LC,1.02,5.0,-0.13,1.37,3.94,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.51,-36.88,110.53,32.89,53867.0,-28472.0,48735.0,-701.92,65.0,L-MC,5.64,259.0,-0.53,0.34,34.21,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.36,-12.97,108.75,81.67,88788.0,-12166.0,81644.0,7338.89,51.0,L-SC,19.29,271.0,-0.14,0.57,58.11,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.00,-11.37,57.45,39.54,51505.0,-11507.0,89652.0,-30.63,68.0,M-SC,6.62,220.0,-0.22,0.63,26.42,AR,ATH,AUTO
51,MASFIN,398.61,-0.88,-6.67,30.78,22.05,28145.0,-6540.0,91440.0,-19.60,45.0,H-SC,6.94,164.0,-0.23,0.64,32.91,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.88,-19.80,53.41,23.03,76705.0,-35455.0,143616.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.50,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-3.60,-10.53,11.79,0.02,18414.0,-18375.0,156184.0,138.28,62.0,M-SC,9.05,234.0,-1.00,1.09,35.85,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.17,0.62,20.20,20.95,51173.0,1566.0,253334.0,-0.35,76.0,X-LC,14.54,31.0,0.03,1.77,36.48,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.51,-36.88,110.53,32.89,53867.0,-28472.0,48735.0,-701.92,65.0,L-MC,5.64,259.0,-0.53,0.34,34.21,XR,NTT,BANKS
0,5PAISA,593.00,-0.57,-34.90,73.44,12.90,104454.0,-76266.0,142230.0,109.13,58.0,H-SC,13.40,161.0,-0.73,0.99,18.26,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,-0.67,6.49,13.40,20.77,33413.0,15204.0,249348.0,-2.78,62.0,X-MC,3.77,62.0,0.46,1.74,13.86,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.80
1,25,44.69
2,50,75.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.59
MC    29.26
LC    25.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.85
X40      21.22
X40N     11.95
XR        9.53
AR        9.12
XY25      9.11
OX40N     7.65
X200      1.87
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.76
X-LC    20.00
M-SC    11.94
X-SC     8.31
H-MC     4.76
H-LC     3.02
L-SC     1.41
M-MC     1.40
M-LC     1.20
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.79,-4.46,38.25
IT,13.78,-14.58,73.73
FINANCE,9.46,-14.34,62.66
MISC,7.43,-21.86,71.79
ELECTRICAL,5.99,-9.45,49.54
PAINTS,5.96,-8.21,24.28
INSURANCE,4.43,0.24,34.92
PHARMA,3.87,-0.55,32.73
AUTO,3.38,-18.67,69.55


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3045075.0,21
XR,1297680.0,13
AR,1267629.0,10
X40,996504.0,14
X40N,736183.0,9
OX40N,695158.0,10
XY25,335633.0,6
SR,279812.0,2
MH,73627.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3549907.0,25
M-SC,1347252.0,15
X-MC,1222813.0,16
X-LC,893494.0,11
X-SC,729437.0,8
H-MC,397124.0,3
L-SC,260478.0,3
H-LC,125500.0,2
M-LC,119170.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216088.0      6
           AR         870548.0      5
           XR         792892.0      7
M-SC       XY24       777536.0      6
X-MC       X40        439089.0      7
X-LC       X40        393352.0      5
X-MC       XY24       361663.0      3
H-SC       OX40N      316940.0      4
X-SC       X40N       291727.0      3
M-SC       OX40N      290435.0      5
H-SC       SR         279812.0      2
X-SC       XY24       273647.0      3
X-MC       X40N       243425.0      4
X-LC       XY24       226906.0      2
H-MC       AR         207889.0      2
X-LC       X40N       201031.0      2
H-MC       XY24       189235.0      1
X-MC       XY25       178636.0      2
L-SC       XR         172695.0      2
X-SC       X40        164063.0      2
M-SC       XR         159056.0      2
           AR         120225.0      2
M-LC       XR         119170.0      1
L-SC       OX40N       87783.0      1
H-SC       MH          73627.0      1
X-LC       XY25        72205.0      2
H-LC       AR          68967.0      1
           X200        56533.0      1
L-MC       XR          53867.0      1
M-MC       XY25        45268.0      1
L-LC       XY25        39524.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
